In [1]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import random
from collections import deque

In [6]:
env = gym.make('CartPole-v0')
env.reset()
while True: 
    action=env.action_space.sample()
    observation, reward, done, info=env.step(action)# take a random action
    
    print(done)
    if done:
        print(done)
        break
    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [14]:
EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995
class dql_model():
    def __init__(self,action_shape,state_shape):
        self.exploration_rate = EXPLORATION_MAX
        self.action_shape=action_shape
        self.state_shape=state_shape
        self.model=Sequential()
        self.model.add(Dense(24,input_dim=self.state_shape,activation='relu'))
        self.model.add(Dense(24,activation='relu'))
        self.model.add(Dense(24,activation='relu'))
        self.model.add(Dense(self.action_shape,activation='linear'))
        
        opt=tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)

        self.model.compile(loss='mse',optimizer=opt)

    def memory_replay(self,memory):
        batch_size=20
        if len(memory)< batch_size:
            return
        batch= random.sample(memory,batch_size)
        for state,next_state, reward, done, info,action in batch:
            q_update= reward
            if not done:
                q_update= reward + 0.9 * np.amax(self.model.predict(next_state)[0])
            q_values=self.model.predict(state)
            q_values[0][action]= q_update
            self.model.fit(state,q_values,verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)
            
    def action_decision(self,state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_shape)
        action=np.argmax(self.model.predict(state)[0])
        return action
        

In [23]:
SEQ_LEN=100
memory=deque(maxlen=SEQ_LEN)
env = gym.make('CartPole-v0')
dql=dql_model(env.action_space.n,4)
for episode in range(100): 
    state=env.reset().reshape(1,-1)
    while True:
        action=dql.action_decision(state)
        next_state, reward, done, info=env.step(action)# take a random action
        next_state=next_state.reshape(1,-1)
        memory.append([state,next_state, reward, done, info,action])
        state=next_state
        if done:
            print("Episode {} finished".format(episode))
            break
        dql.memory_replay(memory)


Episode 0 finished
Episode 1 finished
Episode 2 finished
Episode 3 finished
Episode 4 finished
Episode 5 finished
Episode 6 finished
Episode 7 finished
Episode 8 finished
Episode 9 finished
Episode 10 finished
Episode 11 finished
Episode 12 finished
Episode 13 finished
Episode 14 finished
Episode 15 finished
Episode 16 finished
Episode 17 finished
Episode 18 finished
Episode 19 finished
Episode 20 finished
Episode 21 finished
Episode 22 finished
Episode 23 finished
Episode 24 finished
Episode 25 finished
Episode 26 finished
Episode 27 finished
Episode 28 finished
Episode 29 finished
Episode 30 finished
Episode 31 finished
Episode 32 finished
Episode 33 finished
Episode 34 finished
Episode 35 finished
Episode 36 finished
Episode 37 finished
Episode 38 finished
Episode 39 finished
Episode 40 finished
Episode 41 finished
Episode 42 finished
Episode 43 finished
Episode 44 finished
Episode 45 finished
Episode 46 finished
Episode 47 finished
Episode 48 finished
Episode 49 finished
Episode 50

In [30]:
scores=[]
for episode in range(10): 
    state=env.reset().reshape(1,-1)
    env.render()
    score_acc=0
    while True:
        action=dql.action_decision(state)
        next_state, reward, done, info=env.step(action)# take a random action
        next_state=next_state.reshape(1,-1)
        if done:
            reward=-reward
        memory.append([state,next_state, reward, done, info,action])
        state=next_state
        score_acc+=reward
        if done:
            scores.append(score_acc)
            break
    env.close()

In [31]:
scores

[7.0, 7.0, 7.0, 8.0, 7.0, 8.0, 8.0, 8.0, 10.0, 7.0]

In [32]:
state=env.reset().reshape(1,-1)
dql.model.predict(state)

array([[12.429112, 11.220982]], dtype=float32)